# Fuel Field Observations from Oklahoma

The purpose of this notebook is to clean and format data received from JD Carlson (via Derek Vanderkamp) on fuel moisture field observations conducted in Oklahoma in 1996-1997.

## Background

- Part of publication in 2007
- Used to calibrate Nelson model, used by many agencies

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.utils import time_intp

In [ ]:
df = pd.read_excel("data/oklahoma_Carlson_data.xlsx")

In [ ]:
df

## Explore

- Includes weather data and fuel moisture data.
- Weather data and fuel moisture data not exactly lined up in time
- Separate rows with missing weather or missing fuel moisture if not at the same time


QUESTIONS: 
- What is `esat`?
- Timezone?

GOAL:
NOTE: working just with 1h data now

- Separate weather from FMC data
- Sort by time
- Interpolate weather to line up to exact time of FMC data
- Join back together

In [ ]:
print(f"Unique sites: {len(df.site.unique())}")
print(f"Unique subsites: {len(df.subsite.unique())}")
print(f"Unique res: {len(df.res.unique())}")

In [ ]:
df.columns

In [ ]:
# Define Variable Sets
tvars = ["year", "month", "yday", "mday", "hour", "min", "date"]
wvars = ["k.down", "precip", "rh", "temp", "vap.press", "vpd",
         "wind.speed", "vap.den"]
fvars = ["fuel.mois", "fuel.mois_1000h", "fuel.mois_100h", "fuel.mois_1h"]

### Fix Date
The date column as received in the spreadsheet has a couple of missing dates, and the 0 hour dates are read in oddly. Check both

In [ ]:
# Construct date
dates = pd.to_datetime(dict(
    year=df['year'],
    month=df['month'],
    day=df['mday'],
    hour=df['hour'],
    minute=df['min']
))

print(f"Number of NA Dates: {np.sum(dates.isna())}")

In [ ]:
# Check 0 hour dates
df[df.hour == 0][tvars]

In [ ]:
# Confirm that hour and minute info is in timestamp
print(df[df.hour == 0][tvars].iloc[0])

In [ ]:
print(df[df.hour == 0]['date'].dt.hour.unique())
print(df[df.hour == 0]['date'].dt.minute.unique())

In [ ]:
# Compare to date column in data frame, manually extract
inds = np.where(dates != df.date)[0]
print(f"Number of Date Mismatches: {len(inds)}")
print(f"Number of Missing Dates: {np.sum(df.date.isna())}")

In [ ]:
# Manually Investigate
print(dates.iloc[inds])
df.iloc[inds][tvars]

**NOTE:** the manually constructed date column exists for all but a couple of NA dates in the spreadsheet. We will replace the date column with the manually constructed one to overwrite the two missing dates.

In [ ]:
df.date = dates

print(f"Number of Missing Dates: {np.sum(df.date.isna())}")

### Separate Datasets

Note: filtering FMC data to 1h, as that is the main motivation for collecting this dataset. Ignoring cases where FMC observations exist for other fuel classes for now

In [ ]:
# Extract weather data
weather = df[tvars + wvars]
weather = weather[~(weather.rh.isna()) & ~(weather.temp.isna())]

# Extract fuel data
fm = df[tvars + fvars]
fm = fm[~(fm['fuel.mois_1h'].isna())]

# Sort by time
weather = weather.sort_values("date").reset_index(drop=True)
fm = fm.sort_values("date").reset_index(drop=True)

In [ ]:
# Explore Time
wlag = weather.date.diff()
flag = fm.date.diff()

u = wlag.dropna().unique()
print(f"Weather Time Range:\n    {weather.date.min()} to {weather.date.max()}")
print(f"Weather time increments: {u}")

print()
u = flag.dropna().unique()
print(f"FM Time Range:\n    {fm.date.min()} to {fm.date.max()}")
print(f"FM time increments: ")
print(f"    Min increment: {u.min()}")
print(f"    Max increment: {u.max()}")
print(f"    Mean increment: {u.mean()}")

## Interpolate Weather and Join

Line up weather data with exact time of FM observation with temporal interpolation. Using linear interp for now. Weather is higher resolution in time so linear is sensible. 

Programmatically: 
- set date colunmn to index in weather data
- use numpy interp on 

In [ ]:
def align_weather_to_fm(w2: pd.DataFrame, fm: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
    """
    Align weather data (w2) to the irregular fm timestamps.
    - Numeric columns: linear interpolation in time (inside range only).
    - Non-numeric columns: forward fill then backfill.
    - Returns a DataFrame with rows exactly at fm[date_col] times.
    """
    w2 = w2.copy()
    f = fm[[date_col]].copy()

    # Ensure datetime
    w2[date_col] = pd.to_datetime(w2[date_col], utc=False)
    f[date_col] = pd.to_datetime(f[date_col], utc=False)

    # Drop duplicates and sort
    w2 = w2.drop_duplicates(subset=[date_col]).sort_values(date_col)
    f = f.drop_duplicates(subset=[date_col]).sort_values(date_col)

    # Indexing
    w2 = w2.set_index(date_col)
    fm_times = pd.DatetimeIndex(f[date_col])

    # Union index for interpolation anchors and targets
    combined_idx = w2.index.union(fm_times)

    # Reindex and interpolate
    w2r = w2.reindex(combined_idx)

    num_cols = w2r.select_dtypes(include=[np.number]).columns
    non_num_cols = w2r.columns.difference(num_cols)

    if len(num_cols) > 0:
        w2r[num_cols] = w2r[num_cols].interpolate(method='time', limit_area='inside')

    if len(non_num_cols) > 0:
        w2r[non_num_cols] = w2r[non_num_cols].ffill().bfill()

    # Select rows at fm times
    aligned = w2r.loc[fm_times].copy()
    aligned = aligned.assign(**{date_col: fm_times})
    cols = [date_col] + [c for c in aligned.columns if c != date_col]
    aligned = aligned[cols]

    return aligned

In [ ]:
# Remove extraneous time columns
w2 = weather[['date'] + wvars]
wdata = align_weather_to_fm(w2, fm)

In [ ]:
# Check before merge
print(f"{wdata.shape[0]=}, Matches FM data: {wdata.shape[0] == fm.shape[0]}")
print(f"Date Columns Match: {np.all(wdata.date.values == fm.date.values)}")

In [ ]:
# Merge
final = pd.concat([fm.reset_index(drop=True), wdata.drop(columns=['date']).reset_index(drop=True)], axis=1)
final

In [ ]:
# Check an interpolated time
from IPython.display import display


# Get the FM timestamp at index 0
test_time = final.loc[0, 'date']

print("FM time:", test_time)

# Pull the two nearest weather rows before/after this FM time
mask = (weather['date'] <= test_time)
before = weather.loc[mask].tail(1)
after = weather.loc[~mask].head(1)

print("\nWeather row before/after:")
display(pd.concat([before, after]))

print("\nInterpolated weather row (from final):")
display(final.iloc[[0]])

In [ ]:
# Write Out
final.to_excel("data/carlson_data_cleaned.xlsx", index=False)
weather.to_excel("data/carlson_weather.xlsx", index=False)